In [1]:
from main_agent import Main_agent
import requests
from PIL import Image
from io import BytesIO
import tempfile
import os
from spire.doc import Document,FileFormat
from spire.doc.common import *
import gradio as gr
import pandas as pd


In [2]:
agent=Main_agent()

In [3]:
def temp_image(url:str):
    image=requests.get(url)
    image=Image.open(BytesIO(image.content))
    with tempfile.NamedTemporaryFile(mode='w', suffix='.png', delete=False) as temp_file:
        image.save(temp_file.name)
        image_file_path = temp_file.name
    return image_file_path

In [4]:
def paper_to_markdown(paper:dict) -> str:
    """
    Convert a Paper_layout object into a markdown string.
    
    Args:
        paper (Paper_layout): The paper layout to convert
        
    Returns:
        str: The complete markdown document
    """
    
    markdown = []
    
    # Add title
    markdown.append(paper.get('title'))
    markdown.append("\n\n")
    if paper.get('image_url'):
        markdown.append(f"![Image]({paper.get('image_url')})\n")
        markdown.append("\n\n")
    
    # Add body sections
    for section in paper.get('paragraphs'):
        markdown.append(f"## {section.get('title')}\n")
        if section.get('content'):
            markdown.append(section.get('content'))
            markdown.append("\n")


        markdown.append("\n\n")
    
    if paper.get('table'):
        markdown.append(pd.DataFrame(data=paper.get('table').get('data'), columns=paper.get('table').get('columns')).to_markdown())
        markdown.append("\n\n")
    # Add references
    markdown.append("## References\n")
    markdown.append(str(paper.get('references')))
    
    return "".join(markdown)

In [5]:
def save_markdown_as_docx(markdown:str):
# Create a temporary file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False) as temp_file:
        temp_file.write(markdown)
        temp_file_path = temp_file.name

    # Create a new document and load from the temporary files
    document = Document()
    document.LoadFromFile(temp_file_path)

    # Save as DOCX
    document.SaveToFile("output.docx", FileFormat.Docx)

    # Clean up the temporary file
    os.unlink(temp_file_path)

In [6]:
test_data={'title': 'Sunday App and Stripe: A Payment Strategy for Restaurants',
 'image_url': 'https://www.menutiger.com/_next/image?url=http%3A%2F%2Fcms.menutiger.com%2Fwp-content%2Fuploads%2F2023%2F07%2Fseamless-stripe-payment-integration-for-restaurants-2.jpg&w=2048&q=75',
 'paragraphs': [{'title': 'Introduction',
   'content': 'Sunday is a payment solution designed to streamline the restaurant experience. Co-founded in April 2021 by Christine de Wendel, Victor Lugger, and Tigrane Seydoux, it aims to address inefficiencies in the restaurant industry by simplifying payments through QR code technology, allowing customers to quickly access their bill, split it, add a tip, and pay in seconds. Integrated with existing POS systems and partnering with payment providers like Stripe and Checkout.com, Sunday also offers features such as digital menus with customizable options, order and pay capabilities, and tools to enhance online reputation and customer engagement for restaurants.'},
  {'title': "Sunday's Payment Strategy",
   'content': "Sunday's payment strategy focuses on streamlining the payment process in restaurants through its QR code-based system, enabling customers to pay quickly and efficiently. This approach not only simplifies payments but also reduces processing fees for restaurants through partnerships with companies like Stripe and Checkout.com. The company's payment solution is designed to integrate seamlessly with existing POS systems, offering additional features such as digital menus, order and pay capabilities, and tools to improve online reputation and customer engagement."},
  {'title': 'Partnership with Stripe',
   'content': 'On Wednesday, Sunday announced its new Order & Pay capabilities and its partnership with Stripe, which has allowed it to reduce processing fees for restaurants by an average of 0.5%. Through Stripe, Sunday processes payments via a QR code at the table, which allows customers to pay, split the bill, and add a tip quickly. Sunday is also negotiating with payment providers for better rates.'},
  {'title': 'Stripe Integration and Fee Reduction',
   'content': 'Sunday leverages Stripe to process payments, utilizing a QR code system for customer convenience. This partnership has enabled Sunday to reduce processing fees for restaurants by an average of 0.5%.'},
  {'title': 'Benefits for Restaurants',
   'content': 'Sunday offers numerous benefits for restaurants by leveraging its technology and partnerships, including Stripe. Restaurants using Sunday can experience increased efficiency through streamlined payment processes and order and pay capabilities. Sunday also helps restaurants collect valuable data and analytics, including customer insights and business analytics, which can be used to improve operations and enhance customer experience. Additionally, the integration with Stripe reduces processing fees, benefiting restaurant finances.'},
  {'title': 'Conclusion',
   'content': 'In conclusion, Sunday has significantly impacted restaurant payments by streamlining the payment process and reducing processing fees through its partnership with Stripe. This collaboration has enabled restaurants to enhance efficiency, improve customer experiences, and gain valuable data insights, positioning Sunday as a key player in revolutionizing the restaurant industry.'}],
 'table': {'data': [['App Functionality',
    'Access bill and pay in two clicks, browse the full menu.'],
   ['Payment Processing', 'Payments processed by Stripe.'],
   ['Stripe Agreement Benefit',
    'Reduced processing fees for restaurants by an average of 0.5%.'],
   ['Restaurant Benefits',
    'More tips, more reviews, and more data for restaurants. Increased order and average basket size.'],
   ['Point of Sale Integration',
    'Works seamlessly with existing POS systems.'],
   ['Payment Method', 'Guests scan a QR code to pay.'],
   ['Additional Features',
    'Digital menu with customizable features (scheduling, tags, extras). Options to split the bill and tip.']],
  'columns': ['Aspect', 'Details']},
 'references': 'https://sundayapp.com/2024-recap-how-sunday-revolutionized-restaurant-payments/, https://sundayapp.com/blog/, https://www.checkout.com/case-studies/sunday-brings-the-ease-of-online-payments-to-the-offline-dining-experience'}

In [7]:


def reset():
    agent.reset()

def update_markdown():
    """Get the current research paper markdown"""
    # if agent.deps.deep_search_results:
    #     research_paper = agent.deps.deep_search_results[-1]
    #     if research_paper:
    #         return paper_to_markdown(research_paper)
    return paper_to_markdown(test_data)
    # return "No research paper available yet."

def update_dataframe():
    # if agent.deps.table_data:
    #     return pd.DataFrame(data=agent.deps.table_data.get('data'), columns=agent.deps.table_data.get('columns'))
    # return "No table data available yet."
    return pd.DataFrame(data=test_data.get('table').get('data'), columns=test_data.get('table').get('columns'))



In [14]:
with gr.Blocks() as demo:
    gr.Markdown("# Research Assistant Chatbot")
    
    with gr.Tab():
        with gr.Column():
            chatbot = gr.Chatbot(type='messages')
            msg = gr.Textbox(label="Your Query")
            with gr.Row():
                submit_btn = gr.Button("Submit")
                reset_btn = gr.Button("Reset")

    with gr.Tab("Research Paper"):
        
        markdown_display = gr.Markdown(label="Research Paper")
        
        save_btn = gr.Button("Save as DOCX")
        save_status = gr.Textbox(label="Save Status", interactive=False)
    with gr.Tab("Editable Table"):
        table_display = gr.Dataframe(label="Editable Table")
    
    # Function to handle chat
    async def respond(input, history):
        history=[]
        # response =await agent.chat(input)
        history.append({"role": "user", "content": input})
        # history.append({"role": "assistant", "content": str(response)})
        history.append({"role": "assistant", "content": "test"})
        return"", history, update_markdown(), update_dataframe()
    
    # Function to reset the chat
    def reset():
        agent.reset()
        return [], update_markdown()
    
    # Function to save the current markdown
    def save():
        current_markdown = update_markdown()
        return save_markdown_as_docx(current_markdown)
    
    # Set up event handlers
    msg.submit(respond, [msg, chatbot], [msg,chatbot,markdown_display,table_display])
    submit_btn.click(respond, [msg, chatbot], [chatbot,markdown_display,table_display])
    reset_btn.click(reset, [], [chatbot, markdown_display,table_display])
    save_btn.click(save, [], [save_status])

# Launch the app
if __name__ == "__main__":
    demo.launch(debug=True) 

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
